In [1]:
import numpy as np
import pandas as pd
from bertopic import BERTopic
from transformers import pipeline, AutoTokenizer
from sentence_transformers import SentenceTransformer
from umap import UMAP    
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import sys
import os
sys.path.append(os.path.abspath(".."))
from process.helpers import text_embedding

f:\Python\Setting\Miniconda\envs\feyy311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate embeddings
The article utilizes the 'all-MiniLM-L12-v2' in sentencetransformers as a light and versatile language model for embeddings. Nevertheless, the model is only able to process less than 512 tokens therefore we created a custom function to cut the text into appropriate paragraphs and return the mean value of embeddings of all the paragraphs.

see: https://sbert.net/

“For transformer models like BERT, RoBERTa, DistilBERT etc., the runtime and memory requirement grows quadratic with the input length. This limits transformers to inputs of certain lengths. A common value for BERT-based models are 512 tokens, which corresponds to about 300-400 words (for English).”

In [2]:
# convert the data into list for the model input
news = pd.read_csv("../data/splited/m_ir_ms.csv")
docs = news['content'].astype(str).tolist()

In [3]:
# generate the tokenizer and introduce the model
tokenizer = AutoTokenizer.from_pretrained('../models/all-MiniLM-L12-v2')
embedding_model = SentenceTransformer('../models/all-MiniLM-L12-v2')
max_token = 512

In [4]:
# save the model
embeddings = [text_embedding(doc,tokenizer,embedding_model,max_token) for doc in docs]
embeddings = np.array(embeddings)  # convert into numpy array!
print(type(embeddings),embeddings.shape)
np.save('../data/embs/m_ir_ms_emb.npy',embeddings)

Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Batches: 100%|██████████| 1/1 [00:00<00:00, 20.02it/s]

<class 'numpy.ndarray'> (717, 384)


# BERTopic model
The BERTopic model selects the umap as the dim-decreasing model and hdbscan as clustering model. First we tested the default values of parameters apart from the 'metric = cosine' as it is well recognized to be more appropriate in the text calculating. Then we fine-tuned some parameters to make the topics more scattered.

In [13]:
# 1. load the local embeddings
embeddings = np.load('../data/embs/m_ir_ms_emb.npy')

# 2. create umap model for dim decreasing
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state= 22 # to avoid stochastic problems
)

'''
    n_neighbors: float (optional, default 15)
        The size of local neighborhood (in terms of number of neighboring
        sample points) used for manifold approximation. Larger values
        result in more global views of the manifold, while smaller
        values result in more local data being preserved. In general
        values should be in the range 2 to 100.

    n_components: int (optional, default 2)
        The dimension of the space to embed into. This defaults to 2 to
        provide easy visualization, but can reasonably be set to any
        integer value in the range 2 to 100.

    metric: string or function (optional, default 'euclidean')
        The metric to use to compute distances in high dimensional space.
        If a string is passed it must match a valid predefined metric. If
        a general metric is required a function that takes two 1d arrays and
        returns a float can be provided. For performance purposes it is
        required that this be a numba jit'd function. Valid string metrics
        include:

        * euclidean
        * manhattan
        * chebyshev
        * minkowski
        * canberra
        * braycurtis
        * mahalanobis
        * wminkowski
        * seuclidean
        * cosine
        * correlation
        * haversine
        * hamming
        * jaccard
        * dice
        * russelrao
        * kulsinski
        * ll_dirichlet
        * hellinger
        * rogerstanimoto
        * sokalmichener
        * sokalsneath
        * yule

        Metrics that take arguments (such as minkowski, mahalanobis etc.)
        can have arguments passed via the metric_kwds dictionary. At this
        time care must be taken and dictionary elements must be ordered
        appropriately; this will hopefully be fixed in the future.
        
     min_dist: float (optional, default 0.1)
        The effective minimum distance between embedded points. Smaller values
        will result in a more clustered/clumped embedding where nearby points
        on the manifold are drawn closer together, while larger values will
        result on a more even dispersal of points. The value should be set
        relative to the ``spread`` value, which determines the scale at which
        embedded points will be spread out.
'''

"\n    n_neighbors: float (optional, default 15)\n        The size of local neighborhood (in terms of number of neighboring\n        sample points) used for manifold approximation. Larger values\n        result in more global views of the manifold, while smaller\n        values result in more local data being preserved. In general\n        values should be in the range 2 to 100.\n\n    n_components: int (optional, default 2)\n        The dimension of the space to embed into. This defaults to 2 to\n        provide easy visualization, but can reasonably be set to any\n        integer value in the range 2 to 100.\n\n    metric: string or function (optional, default 'euclidean')\n        The metric to use to compute distances in high dimensional space.\n        If a string is passed it must match a valid predefined metric. If\n        a general metric is required a function that takes two 1d arrays and\n        returns a float can be provided. For performance purposes it is\n        requir

In [26]:

# 3. create HBBSCAN model for clustering
# to decrease the outliers，fine-tune min_cluster_size  and min_samples
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=10,
  min_samples=9,
  metric='euclidean'
)

'''
  min_cluster_size : int, optional (default=5)
        The minimum size of clusters; single linkage splits that contain
        fewer points than this will be considered points "falling out" of a
        cluster rather than a cluster splitting into two new clusters.

    min_samples : int, optional (default=None)
        The number of samples in a neighborhood for a point to be considered as a core point.
	      This includes the point itself. When None, defaults to min_cluster_size.

    metric : string, or callable, optional (default='euclidean')
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string or callable, it must be one of
        the options allowed by metrics.pairwise.pairwise_distances for its
        metric parameter.
        If metric is "precomputed", X is assumed to be a distance matrix and
        must be square.

'''

'\n  min_cluster_size : int, optional (default=5)\n        The minimum size of clusters; single linkage splits that contain\n        fewer points than this will be considered points "falling out" of a\n        cluster rather than a cluster splitting into two new clusters.\n\n    min_samples : int, optional (default=None)\n        The number of samples in a neighborhood for a point to be considered as a core point.\n\t      This includes the point itself. When None, defaults to min_cluster_size.\n\n    metric : string, or callable, optional (default=\'euclidean\')\n        The metric to use when calculating distance between instances in a\n        feature array. If metric is a string or callable, it must be one of\n        the options allowed by metrics.pairwise.pairwise_distances for its\n        metric parameter.\n        If metric is "precomputed", X is assumed to be a distance matrix and\n        must be square.\n\n'

In [29]:
# 5. add stop words
custom_words =  ['irish','ireland','said','government','ms','party', 'mr','000', 'yes']
stop_words = list(text.ENGLISH_STOP_WORDS.union(custom_words)) 
vectorizer_model = CountVectorizer(stop_words=stop_words) # can only pass list/None/'english'
# vectorizer_model = CountVectorizer(stop_words='english')

# 6. combine the topic model
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

In [30]:
# 7. inspect the topics
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,131,-1_people_says_minister_election,"[people, says, minister, election, fine, gael,...",[Who will be Ireland’s first directly elected ...
1,0,144,0_tax_minister_housing_year,"[tax, minister, housing, year, new, million, b...",[Decision on Apple windfall spending must be ...
2,1,126,1_israel_gaza_israeli_palestine,"[israel, gaza, israeli, palestine, palestinian...",[‘This is not how we wanted to recognise Pales...
3,2,73,2_election_fine_gael_fianna,"[election, fine, gael, fianna, fáil, general, ...",[General election could take place in October ...
4,3,45,3_mcgrath_commissioner_leyen_der,"[mcgrath, commissioner, leyen, der, von, europ...",[Government to make last minute lobbying push ...
5,4,43,4_asylum_accommodation_seekers_people,"[asylum, accommodation, seekers, people, prote...",[Analysis: What effect will the Government’s s...
6,5,28,5_harris_taoiseach_simon_election,"[harris, taoiseach, simon, election, fine, gae...",[Simon Harris rules out early election after t...
7,6,27,6_northern_uk_dup_stormont,"[northern, uk, dup, stormont, legacy, belfast,...",[Varadkar warns over previous ‘false dawns’ of...
8,7,24,7_care_referendum_family_referendums,"[care, referendum, family, referendums, women,...",[Referendum profiles: The No campaign’s key pl...
9,8,20,8_varadkar_taoiseach_fine_gael,"[varadkar, taoiseach, fine, gael, time, leo, l...",[Fifteen things to expect in Leo Varadkar’s au...


# Visualization

In [31]:
topic_model.visualize_barchart()

In [32]:
topic_model.visualize_topics()

In [33]:
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

In [34]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 12/12 [00:00<00:00, 162.69it/s]


In [35]:
topic_model.merge_topics(docs, 
[
 [9,4,0,10,7],
 [11,2,5,8,3],
 [1,12,6]
 
]
)
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,131,-1_people_minister_says_election,"[people, minister, says, election, fine, gael,...",[Who will be Ireland’s first directly elected ...
1,0,242,0_people_minister_tax_state,"[people, minister, tax, state, public, housing...","[Fine Gael pledges to deliver over 300,000 hom..."
2,1,179,1_election_fine_gael_taoiseach,"[election, fine, gael, taoiseach, european, fi...",[Jack Chambers named as new Minister for Finan...
3,2,165,2_israel_gaza_israeli_martin,"[israel, gaza, israeli, martin, palestine, sta...",[‘This is not how we wanted to recognise Pales...


In [36]:
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

In [37]:
# 8. save the result
topic_docs = topic_model.get_document_info(docs)
topic_docs.to_csv('../data/clusters/cluster_m_ir_ms.csv', encoding='utf-8-sig', index=False)